In [55]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels import api
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS
import seaborn as sns
from scipy.optimize import minimize
import scipy.stats as stats
import pymc3 as pm3
import numdifftools as ndt
from statsmodels.base.model import GenericLikelihoodModel


In [56]:
# Loading data
df = pd.read_stata("PS4_data.dta")
df.shape
df.size
df.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


In [64]:
# Filtering data such that observations only include male heads of households whose age is between 25 and 60 and hourly wage is greater than $7.
dff = df[(df['hannhrs']>0)&(df['hsex']==1)&(df['age']>25)&(df['age']<60)]
dff['wph'] = dff['hlabinc']/dff['hannhrs']
dff=dff[dff['wph']>7]
dff.head()
dff.describe()

<ipython-input-64-214c33507ae1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['wph'] = dff['hlabinc']/dff['hannhrs']


,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce,wph
count,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,5.747700e+04,37442.000000,57477.000000,51988.000000,57381.000000,...,57477.0,50180.0,57477.000000,50180.000000,57477.000000,50180.000000,57097.000000,49825.000000,57477.000000,57477.000000
mean,1507.533935,1986.635245,3484.484507,2228.364990,994.471619,5.280176e+04,23166.681641,1.173026,1.086905,1.102630,...,1.0,2.0,39.224247,37.308151,59.409206,67.729515,13.529993,13.212443,0.617346,24.306034
std,828.790640,8.744894,2254.476920,619.743286,927.341492,5.228071e+04,21082.244141,1.216322,0.345212,0.371816,...,0.0,0.0,9.579065,9.563719,78.759903,81.602432,2.449510,2.167871,0.206892,25.154028
min,1.000000,1971.000000,1.000000,2.000000,0.000000,1.666980e+01,1.192780,0.000000,1.000000,1.000000,...,1.0,2.0,25.000000,16.000000,1.000000,1.000000,1.000000,1.000000,0.247121,7.000252
25%,782.000000,1979.000000,1694.000000,1952.000000,0.000000,3.035003e+04,8926.479492,0.000000,1.000000,1.000000,...,1.0,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.421747,13.947624
50%,1542.000000,1987.000000,3301.000000,2160.000000,960.000000,4.377034e+04,19527.019531,1.000000,1.000000,1.000000,...,1.0,2.0,38.000000,36.000000,4.000000,4.000000,13.000000,12.000000,0.635834,19.905161
75%,2225.000000,1994.000000,5006.000000,2517.000000,1904.000000,6.136955e+04,31881.078125,2.000000,1.000000,1.000000,...,1.0,2.0,47.000000,44.000000,170.000000,170.000000,16.000000,15.000000,0.803488,27.787226
max,2930.000000,2002.000000,16968.000000,5840.000000,5840.000000,3.771521e+06,417271.468750,11.000000,8.000000,3.000000,...,1.0,2.0,60.000000,70.000000,227.000000,230.000000,17.000000,17.000000,0.928007,1717.330322


In [93]:
#Creating a logged hourly wage variable
dff['logwph']=np.log(dff['wph'])
#Creating a age squared variable
dff['age_sq']=np.square(dff['age'])
#Creating a constant variable '1' for ols intercept
dff['constant']=1
dff

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,hpersno,wpersno,hyrsed,wyrsed,pce,wph,logwage,age_sq,logwph,constant
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,1.0,2.0,12.0,12.0,0.247121,41.318916,3.721320,2601.0,3.721320,1
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,1.0,2.0,5.0,5.0,0.247121,11.274115,2.422509,3025.0,2.422509,1
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,1.0,2.0,16.0,12.0,0.247121,10.257995,2.328057,625.0,2.328057,1
11173,284,1971,20,Head,2400.0,0.0,76885.437500,NaN,2,1.0,...,1.0,2.0,16.0,12.0,0.247121,32.035599,3.466848,1521.0,3.466848,1
11175,50,1971,29,Head,3164.0,2000.0,31968.156250,28326.214844,3,1.0,...,1.0,2.0,12.0,12.0,0.247121,10.103716,2.312903,1296.0,2.312903,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123772,514,2002,7842,Head,2285.0,1904.0,63577.074219,37715.214844,0,2.0,...,170.0,4.0,14.0,14.0,0.928007,27.823666,3.325887,3136.0,3.325887,1
123775,1016,2002,7847,Head,16.0,0.0,4310.310059,NaN,0,NaN,...,5.0,NaN,16.0,NaN,0.928007,269.394379,5.596177,1296.0,5.596177,1
123777,2678,2002,7850,Head,2080.0,0.0,34482.480469,NaN,0,1.0,...,171.0,176.0,12.0,14.0,0.928007,16.578115,2.808084,2601.0,2.808084,1
123784,1634,2002,7868,Head,2288.0,0.0,42025.523438,NaN,0,1.0,...,4.0,NaN,12.0,NaN,0.928007,18.367798,2.910599,2209.0,2.910599,1


In [94]:
#Given the above descriptive statistics, I find that available race information is only White, Black, and Native. So I treat Native as 'Otherrace' and build a linear model without 'Hispanic' variable unlike the homework instruction.
#Creating race dummies
dfff=pd.get_dummies(dff, columns = ['hrace'], drop_first=True)


In [95]:
dfff=pd.get_dummies(dff, columns = ['hrace'], drop_first=True)
dfff.rename(columns={'hrace_2.0':'black','hrace_3.0':'otherrace' }, inplace=True)
dfff.head(n=20)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,hyrsed,wyrsed,pce,wph,logwage,age_sq,logwph,constant,black,otherrace
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,12.0,12.0,0.247121,41.318916,3.721320,2601.0,3.721320,1,0,0
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,5.0,5.0,0.247121,11.274115,2.422509,3025.0,2.422509,1,0,0
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,16.0,12.0,0.247121,10.257995,2.328057,625.0,2.328057,1,0,0
11173,284,1971,20,Head,2400.0,0.0,76885.437500,NaN,2,1.0,...,16.0,12.0,0.247121,32.035599,3.466848,1521.0,3.466848,1,0,0
11175,50,1971,29,Head,3164.0,2000.0,31968.156250,28326.214844,3,1.0,...,12.0,12.0,0.247121,10.103716,2.312903,1296.0,2.312903,1,0,0
11192,1064,1971,56,Head,980.0,72.0,18614.369141,4451.262207,1,1.0,...,17.0,16.0,0.247121,18.994255,2.944137,961.0,2.944137,1,0,0
11196,1679,1971,61,Head,2652.0,1960.0,25461.220703,16101.429688,1,2.0,...,11.0,9.0,0.247121,9.600762,2.261842,1764.0,2.261842,1,1,0
11198,1415,1971,63,Head,2416.0,0.0,39980.429688,NaN,3,1.0,...,11.0,11.0,0.247121,16.548191,2.806277,1296.0,2.806277,1,0,0
11199,280,1971,64,Head,3427.0,1040.0,56611.960938,7688.543945,2,1.0,...,9.0,12.0,0.247121,16.519394,2.804535,1936.0,2.804535,1,0,0
11202,1360,1971,67,Head,2500.0,1760.0,50582.523438,7122.019531,1,NaN,...,7.0,7.0,0.247121,20.233009,3.007315,3025.0,3.007315,1,0,0


In [75]:
dfff.describe()


,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hafdc,...,wpersno,hyrsed,wyrsed,pce,wph,logwage,age_sq,logwph,black,otherrace
count,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,5.747700e+04,37442.000000,57477.000000,51988.000000,204.000000,...,50180.000000,57097.000000,49825.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000
mean,1507.533935,1986.635245,3484.484507,2228.364990,994.471619,5.280176e+04,23166.681641,1.173026,1.086905,2454.374512,...,67.729515,13.529993,13.212443,0.617346,24.306034,3.010414,1629.789673,3.010414,0.056388,0.023035
std,828.790640,8.744894,2254.476920,619.743286,927.341492,5.228071e+04,21082.244141,1.216322,0.345212,2121.048096,...,81.602432,2.449510,2.167871,0.206892,25.154028,0.543891,791.694885,0.543891,0.230671,0.150017
min,1.000000,1971.000000,1.000000,2.000000,0.000000,1.666980e+01,1.192780,0.000000,1.000000,86.000000,...,1.000000,1.000000,1.000000,0.247121,7.000252,1.945946,625.000000,1.945946,0.000000,0.000000
25%,782.000000,1979.000000,1694.000000,1952.000000,0.000000,3.035003e+04,8926.479492,0.000000,1.000000,874.000000,...,2.000000,12.000000,12.000000,0.421747,13.947624,2.635309,961.000000,2.635309,0.000000,0.000000
50%,1542.000000,1987.000000,3301.000000,2160.000000,960.000000,4.377034e+04,19527.019531,1.000000,1.000000,1836.000000,...,4.000000,13.000000,12.000000,0.635834,19.905161,2.990979,1444.000000,2.990979,0.000000,0.000000
75%,2225.000000,1994.000000,5006.000000,2517.000000,1904.000000,6.136955e+04,31881.078125,2.000000,1.000000,3370.000000,...,170.000000,16.000000,15.000000,0.803488,27.787226,3.324576,2209.000000,3.324576,0.000000,0.000000
max,2930.000000,2002.000000,16968.000000,5840.000000,5840.000000,3.771521e+06,417271.468750,11.000000,8.000000,12000.000000,...,230.000000,17.000000,17.000000,0.928007,1717.330322,7.448526,3600.000000,7.448526,1.000000,1.000000


In [120]:
#Keep only dependent and indepdent variables of interest in the data set in use.
dd=dfff[['year','logwph','hyrsed', 'age','age_sq','black','otherrace','constant']]
dd.head()
dd.describe()

,year,logwph,hyrsed,age,age_sq,black,otherrace,constant
count,57477.000000,57477.000000,57097.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.0
mean,1986.635245,3.010414,13.529993,39.224247,1629.789673,0.056388,0.023035,1.0
std,8.744894,0.543891,2.449510,9.579065,791.694885,0.230671,0.150017,0.0
min,1971.000000,1.945946,1.000000,25.000000,625.000000,0.000000,0.000000,1.0
25%,1979.000000,2.635309,12.000000,31.000000,961.000000,0.000000,0.000000,1.0
50%,1987.000000,2.990979,13.000000,38.000000,1444.000000,0.000000,0.000000,1.0
75%,1994.000000,3.324576,16.000000,47.000000,2209.000000,0.000000,0.000000,1.0
max,2002.000000,7.448526,17.000000,60.000000,3600.000000,1.000000,1.000000,1.0


In [151]:
ddd =dd.dropna()
ddd.describe()


,year,logwph,hyrsed,age,age_sq,black,otherrace,constant
count,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.000000,57097.0
mean,1986.584129,3.010798,13.529993,39.242939,1631.272217,0.056343,0.022506,1.0
std,8.716500,0.544119,2.449510,9.579581,791.988159,0.230584,0.148322,0.0
min,1971.000000,1.945946,1.000000,25.000000,625.000000,0.000000,0.000000,1.0
25%,1979.000000,2.635515,12.000000,31.000000,961.000000,0.000000,0.000000,1.0
50%,1987.000000,2.991457,13.000000,38.000000,1444.000000,0.000000,0.000000,1.0
75%,1994.000000,3.324793,16.000000,47.000000,2209.000000,0.000000,0.000000,1.0
max,2002.000000,7.448526,17.000000,60.000000,3600.000000,1.000000,1.000000,1.0



I define the log-likelihood function that needs to be maximized in order to obtain the parameter estimates of the model.
I assume that the error term independtly and identically normally distributed. So,the probability density function is as follows:

$ f(x|\mu,\sigma^2)=\left(\frac{1}{2\pi\sigma^2}\right)^{n/2}\exp\left(-\frac{(x-\mu)^{2}}{2\sigma^2}\right) $

If the residual is substituted into the equation,

$ f(r|0,\sigma^2)=\left(\frac{1}{2\pi\sigma^2}\right)^{n/2}\exp\left(-\frac{(y-x\beta)^{2}}{2\sigma^2}\right) $

The likelihood function L is defined as

$ L(\beta|x_1,x_2,x_3,x_4,x_5,\sigma^2) = \prod\limits_{i=1}^N f(x_i|\beta) $

Then, taking the log of the function, I get

$ l(\beta|x_1,x_2,x_3,x_4,x_5,\sigma^2) = \sum\limits_{i=1}^N \ln f(x_i|\beta) = \sum\limits_{i=1}^N \ln\left(\frac{1}{2\pi\sigma^2}\right)^{1/2}\exp\left(-\frac{(y_i-x_i\beta)^{2}}{2\sigma^2}\right) = \frac{N}{2} \ln\left(\frac{1}{2\pi\sigma^2}\right) - \frac{1}{2\sigma^2}\sum\limits_{i=1}^N\left(y_i- x_i\beta\right)^2 $

In [137]:
#Define the log-likelihood function to be maximized to get the parameter estimates of the model.

def myfunc(parameters, yearly_data):
    '''
    Calculate the log-likelihood function value that is to be maximized
    parameters: A length 6 tuple, model parameters (sigsq, beta0, beta1, beta2, beta3, beta5)
    yearly_data: Data of the particular year to be used for parameter estimation
        
    Return: ll(Value of the log-likelihood function)
    '''
    
    sigsq, beta0, beta1, beta2, beta3, beta4, beta5 = parameters
    
    SSE = ((yearly_data['logwph'] - beta0 - (beta1 * yearly_data['hyrsed'] + beta2 * yearly_data['age'] +
                                             beta3 * yearly_data['age_sq'] +beta4*yearly_data['black']+ beta5 * yearly_data['otherrace'])) ** 2).sum()
    
    n = len(yearly_data)
    
#Calculate the log-likelihood
#Use the negative log-likelihood for a minimization routine
    
    ll = -((n / 2) * (np.log(1 / 2 * np.pi * sigsq)) - (1 / 2 * sigsq) * SSE)
    
    return ll

In [138]:
#Estimation when t = 1971
#Keep data whose year is equal to 1971
df1971 = ddd[(ddd['year'] ==1971)]
#To get reasonable initial values, I run OLS.
reg1 = sm.OLS(endog=df1971['logwph'], exog=df1971[['hyrsed', 'age','age_sq','black','otherrace','constant']], missing = 'drop')
results = reg1.fit()
print(results.params)



hyrsed       0.066500
age          0.064904
age_sq      -0.000617
black       -0.164137
otherrace    0.017512
constant     0.586331
dtype: float64


In [139]:
#For the initial sigma squared value, I use residual standard error. 
sigsq_start=np.sqrt(np.sum(results.resid**2)/results.df_resid)
beta0_start=results.params.constant
beta1_start=results.params.hyrsed      
beta2_start=results.params.age
beta3_start=results.params.age_sq
beta4_start=results.params.black
beta5_start=results.params.otherrace
parameter_start =(sigsq_start,beta0_start,beta1_start,beta2_start,beta3_start,beta4_start,beta5_start)


In [140]:
# Use Nelder-Mead to minimize the negavie value of the LL function.
ll_min_71 = minimize(myfunc, parameter_start, args=(df1971), method="Nelder-Mead",tol=1e-15)

In [141]:
#For the initial sigma squared value, I use residual standard error. 
sigsq_start=np.sqrt(np.sum(results.resid**2)/results.df_resid)
beta0_start=results.params.constant
beta1_start=results.params.hyrsed      
beta2_start=results.params.age
beta3_start=results.params.age_sq
beta4_start=results.params.black
beta5_start=results.params.otherrace
parameter_start =(sigsq_start,beta0_start,beta1_start,beta2_start,beta3_start,beta4_start,beta5_start)


print('The estimates for 1971 are')
print('beta0 = ', ll_min_71['x'][1])
print('beta1 = ', ll_min_71['x'][2])
print('beta2 = ', ll_min_71['x'][3])
print('beta3 = ', ll_min_71['x'][4])
print('beta5 = ', ll_min_71['x'][5])

The estimates for 1971 are
beta0 =  0.5910990581115683
beta1 =  0.06617115124375217
beta2 =  0.06516101151736217
beta3 =  -0.0006206308901083636
beta5 =  -0.16346806488689647


Assuming every other being constant, when the education year of the male head of a household increases by one year, his hourly wage increases by 6.61% on average in 1971. This is slighly smaller than the corresponding OLS estimate (6.65%)

In [145]:
#Estimation when t = 1980
#Keep data whose year is equal to 1980
df1980 = ddd[(ddd['year'] ==1980)]
#To get reasonable initial values, I run OLS.
reg1 = sm.OLS(endog=df1980['logwph'], exog=df1980[['hyrsed', 'age','age_sq','black','otherrace','constant']], missing = 'drop')
results = reg1.fit()
print(results.params)
#For the initial sigma squared value, I use residual standard error. 
sigsq_start=np.sqrt(np.sum(results.resid**2)/results.df_resid)
beta0_start=results.params.constant
beta1_start=results.params.hyrsed      
beta2_start=results.params.age
beta3_start=results.params.age_sq
beta4_start=results.params.black
beta5_start=results.params.otherrace
parameter_start =(sigsq_start,beta0_start,beta1_start,beta2_start,beta3_start,beta4_start,beta5_start)
# Use Nelder-Mead to minimize the negavie value of the LL function.
ll_min_80 = minimize(myfunc, parameter_start, args=(df1980), method="Nelder-Mead",tol=1e-15)
print('The estimates for 1980 are')
print('beta0 = ', ll_min_80['x'][1])
print('beta1 = ', ll_min_80['x'][2])
print('beta2 = ', ll_min_80['x'][3])
print('beta3 = ', ll_min_80['x'][4])
print('beta5 = ', ll_min_80['x'][5])

hyrsed       0.066003
age          0.045569
age_sq      -0.000399
black       -0.103028
otherrace    0.012315
constant     1.002272
dtype: float64
The estimates for 1971 are
beta0 =  1.0012773782943647
beta1 =  0.06598884986677941
beta2 =  0.04562559142809669
beta3 =  -0.00040014406846417276
beta5 =  -0.10352584883534172


Assuming every other being constant, when the education year of the male head of a household increases by one year, his hourly wage increases by 6.59% on average in 1980. Compared to 1971, the increase rate slightly reduces. This is slighly smaller than the corresponding OLS estimate (6.66%)

In [147]:
#Estimation when t = 1990
#Keep data whose year is equal to 1990
df1990 = ddd[(ddd['year'] ==1990)]
#To get reasonable initial values, I run OLS.
reg1 = sm.OLS(endog=df1990['logwph'], exog=df1990[['hyrsed', 'age','age_sq','black','otherrace','constant']], missing = 'drop')
results = reg1.fit()
print(results.params)
#For the initial sigma squared value, I use residual standard error. 
sigsq_start=np.sqrt(np.sum(results.resid**2)/results.df_resid)
beta0_start=results.params.constant
beta1_start=results.params.hyrsed      
beta2_start=results.params.age
beta3_start=results.params.age_sq
beta4_start=results.params.black
beta5_start=results.params.otherrace
parameter_start =(sigsq_start,beta0_start,beta1_start,beta2_start,beta3_start,beta4_start,beta5_start)
# Use Nelder-Mead to minimize the negavie value of the LL function.
ll_min_90 = minimize(myfunc, parameter_start, args=(df1990), method="Nelder-Mead",tol=1e-15)
print('The estimates for 1990 are')
print('beta0 = ', ll_min_90['x'][1])
print('beta1 = ', ll_min_90['x'][2])
print('beta2 = ', ll_min_90['x'][3])
print('beta3 = ', ll_min_90['x'][4])
print('beta5 = ', ll_min_90['x'][5])

hyrsed       0.095499
age          0.057864
age_sq      -0.000540
black       -0.167988
otherrace   -0.052001
constant     0.277243
dtype: float64
The estimates for 1990 are
beta0 =  0.10446906865972849
beta1 =  0.09657917572223035
beta2 =  0.0655425302402351
beta3 =  -0.0006290517340535758
beta5 =  -0.0916117148682842


Assuming every other being constant, when the education year of the male head of a household increases by one year, his hourly wage increases by 9.66% on average in 1990. Compared to the previous years, the increase rate strikingly increases. Even unlike the previous years, this is slighly greater than the corresponding OLS estimate (9.55%)

In [149]:
#Estimation when t = 2000
#Keep data whose year is equal to 2000
df2000= ddd[(ddd['year'] ==2000)]
#To get reasonable initial values, I run OLS.
reg1 = sm.OLS(endog=df2000['logwph'], exog=df2000[['hyrsed', 'age','age_sq','black','otherrace','constant']], missing = 'drop')
results = reg1.fit()
print(results.params)
#For the initial sigma squared value, I use residual standard error. 
sigsq_start=np.sqrt(np.sum(results.resid**2)/results.df_resid)
beta0_start=results.params.constant
beta1_start=results.params.hyrsed      
beta2_start=results.params.age
beta3_start=results.params.age_sq
beta4_start=results.params.black
beta5_start=results.params.otherrace
parameter_start =(sigsq_start,beta0_start,beta1_start,beta2_start,beta3_start,beta4_start,beta5_start)
# Use Nelder-Mead to minimize the negavie value of the LL function.
ll_min_20 = minimize(myfunc, parameter_start, args=(df2000), method="Nelder-Mead",tol=1e-15)
print('The estimates for df2000 are')
print('beta0 = ', ll_min_20['x'][1])
print('beta1 = ', ll_min_20['x'][2])
print('beta2 = ', ll_min_20['x'][3])
print('beta3 = ', ll_min_20['x'][4])
print('beta5 = ', ll_min_20['x'][5])

hyrsed       0.110333
age          0.084360
age_sq      -0.000887
black       -0.259617
otherrace   -0.062144
constant    -0.293275
dtype: float64
The estimates for df2000 are
beta0 =  -0.2955143293603185
beta1 =  0.11049741122286011
beta2 =  0.08441137149891628
beta3 =  -0.0008873943297664032
beta5 =  -0.25845996529875725


Assuming every other being constant, when the education year of the male head of a household increases by one year, his hourly wage increases by 11.05% on average in 2000. Compared to the right previous year, the increase rate considerably increases as well. Likewise this is slighly greater than the corresponding OLS estimate (11.03%)